> ⚠️ **작업 중 (Work in Progress)**: 이 노트북은 현재 개발 중입니다. 일부 코드가 불완전하거나 변경될 수 있습니다.

## 📋 목차

- [워크플로우 개요](#워크플로우-개요)
- [Sequential Workflow](#sequential-workflow)
- [Group Chat Workflow](#group-chat-workflow)
- [Human-in-loop Workflow](#human-in-loop-workflow)

## 🎯 학습 목표

- Microsoft Foundry 워크플로우의 핵심 개념 이해
- Sequential Workflow를 통한 순차적 작업 흐름 구축
- Group Chat Workflow를 통한 다중 에이전트 협업 구현
- Human-in-loop 패턴을 통한 사람 개입 지점 설정
- 워크플로우 배포 및 프로그래매틱 호출

## ⏱️ 예상 소요 시간

약 20분

## 환경 설정

워크플로우 실행을 위한 설정입니다.

In [ ]:
# 환경 변수 로드
import json
import os
import subprocess

# PATH 환경변수 설정 (Azure CLI를 찾을 수 있도록)
possible_paths = [
    "/opt/homebrew/bin",  # macOS (Apple Silicon)
    "/usr/local/bin",     # macOS (Intel) / Linux
    "/usr/bin",           # Linux / GitHub Codespaces
    "/home/linuxbrew/.linuxbrew/bin"  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
except:
    pass

paths_to_add = []
if az_path and az_path not in os.environ.get("PATH", ""):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get("PATH", ""):
            paths_to_add.append(path)

if paths_to_add:
    new_path = ":".join(paths_to_add) + ":" + os.environ.get("PATH", "")
    os.environ["PATH"] = new_path

# 이전 노트북에서 저장한 설정 파일 로드
config_file = ".foundry_config.json"
try:
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    # 환경 변수 설정
    FOUNDRY_NAME = config.get("FOUNDRY_NAME")
    RESOURCE_GROUP = config.get("RESOURCE_GROUP")
    LOCATION = config.get("LOCATION")
    TENANT_ID = config.get("TENANT_ID")
    PROJECT_NAME = config.get("PROJECT_NAME", "proj-default")
    PROJECT_ENDPOINT = config.get("FOUNDRY_ENDPOINT")
    
    # 환경 변수로도 설정 (다른 도구들이 사용할 수 있도록)
    os.environ["FOUNDRY_NAME"] = FOUNDRY_NAME
    os.environ["LOCATION"] = LOCATION
    os.environ["RESOURCE_GROUP"] = RESOURCE_GROUP
    os.environ["AZURE_SUBSCRIPTION_ID"] = config.get("AZURE_SUBSCRIPTION_ID", "")
    os.environ["_ENDPOINT"] = PROJECT_ENDPOINT
    
    print(f"✅ 설정 파일 '{config_file}'에서 환경 변수를 로드했습니다.")
    print(f"\n📌 Foundry Name: {FOUNDRY_NAME}")
    print(f"📌 Resource Group: {RESOURCE_GROUP}")
    print(f"📌 Location: {LOCATION}")
    print(f"📌 프로젝트 엔드포인트: {PROJECT_ENDPOINT}")
    
except FileNotFoundError:
    print(f"⚠️ '{config_file}' 파일을 찾을 수 없습니다.")
    print("💡 01-setup.ipynb를 먼저 실행하여 환경을 설정하세요.")
    raise

# 필수 패키지 설치
%pip install -q azure-ai-projects azure-identity

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

print(f"\n💡 사용할 프로젝트 엔드포인트: {PROJECT_ENDPOINT}")

## Sequential Workflow용 에이전트 생성

Sequential Workflow에서 사용할 에이전트들을 생성합니다.
- **TravelPlannerAgent**: 여행 목적지와 일정을 기획
- **LocalAgent**: 현지 정보를 추가 (Web Search 사용)
- **TravelSummaryAgent**: 최종 요약 및 체크리스트 생성

In [ ]:
# TravelPlannerAgent 생성
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

TRAVEL_PLANNER_INSTRUCTIONS = """당신은 여행 계획 전문가입니다.

역할:
1. 사용자의 여행 요구사항을 분석합니다
2. 목적지의 주요 관광지, 맛집, 숙소를 추천합니다
3. 일자별 여행 일정을 구체적으로 작성합니다
4. 예상 비용과 준비물을 제시합니다

출력 형식:
- 목적지 개요
- 일자별 일정 (아침/점심/저녁 활동)
- 추천 숙소
- 예상 비용
- 준비물 목록

다음 에이전트에게 넘길 정보: 전체 여행 계획"""

agent_travel_planner = client.agents.create_agent(
    model="gpt-5.1",
    name="TravelPlannerAgent",
    instructions=TRAVEL_PLANNER_INSTRUCTIONS
)

print(f"✅ TravelPlannerAgent 생성 완료!")
print(f"   ID: {agent_travel_planner.id}")
print(f"   Name: {agent_travel_planner.name}")

In [ ]:
# LocalAgent 생성 (Web Search 도구 사용)
LOCAL_AGENT_INSTRUCTIONS = """당신은 현지 정보 전문가입니다.

역할:
1. 이전 에이전트의 여행 계획을 받습니다
2. Web search를 사용하여 최신 현지 정보를 검색합니다
3. 실시간 정보를 추가합니다:
   - 현재 날씨 및 기후
   - 현지 축제 및 이벤트
   - 교통 정보 (노선, 요금, 소요시간)
   - 영업시간 및 예약 정보
   - 현지 문화 및 주의사항

출력 형식:
- 원래 일정 + 현지 정보 보강
- 교통편 상세 정보
- 예약 필요 장소 목록
- 현지 팁

다음 에이전트에게 넘길 정보: 현지 정보가 추가된 여행 계획"""

agent_local = client.agents.create_agent(
    model="gpt-5.1",
    name="LocalAgent",
    instructions=LOCAL_AGENT_INSTRUCTIONS,
    tools=[{"type": "web_search"}]
)

print(f"✅ LocalAgent 생성 완료!")
print(f"   ID: {agent_local.id}")
print(f"   Tools: web_search")

In [ ]:
# TravelSummaryAgent 생성
TRAVEL_SUMMARY_INSTRUCTIONS = """당신은 여행 계획 정리 전문가입니다.

역할:
1. 이전 에이전트들의 정보를 종합합니다
2. 실행 가능한 최종 계획으로 정리합니다
3. 체크리스트를 생성합니다

출력 형식:
📋 여행 요약
- 목적지: 
- 기간:
- 예산:

📅 일정 요약 (한눈에 보는 일정)

✅ 출발 전 체크리스트
- [ ] 항목1
- [ ] 항목2

🎒 준비물 체크리스트

📞 긴급 연락처 및 유용한 정보

최종 출력: 프린트 가능한 여행 가이드"""

agent_travel_summary = client.agents.create_agent(
    model="gpt-5.1",
    name="TravelSummaryAgent",
    instructions=TRAVEL_SUMMARY_INSTRUCTIONS
)

print(f"✅ TravelSummaryAgent 생성 완료!")
print(f"   ID: {agent_travel_summary.id}")

## Group Chat Workflow용 에이전트 생성

Group Chat Workflow에서 사용할 에이전트들을 생성합니다.
- **StudentAgent**: 질문에 답변하는 학생 역할
- **TeacherAgent**: 답변을 평가하고 피드백을 주는 교사 역할

In [ ]:
# StudentAgent 생성
STUDENT_INSTRUCTIONS = """너는 문제에 대답하는 에이전트야. 질문이 오면, 항상 답변해줘.

역할:
1. 사용자의 질문을 이해하고 답변을 생성합니다
2. 첫 번째 시도에서는 기본적인 답변을 제공합니다
3. TeacherAgent의 피드백을 받아 답변을 개선합니다
4. 모든 요구사항이 충족될 때까지 답변을 수정합니다

답변 시 고려사항:
- 일정 (날짜, 시간)
- 비용 (예산, 가격)
- 취향 (선호도, 스타일)
- 제약사항 (제한사항, 조건)

개선이 필요하면 TeacherAgent의 피드백을 반영하여 답변을 보완합니다."""

agent_student = client.agents.create_agent(
    model="gpt-5.1",
    name="StudentAgent",
    instructions=STUDENT_INSTRUCTIONS
)

print(f"✅ StudentAgent 생성 완료!")
print(f"   ID: {agent_student.id}")

In [ ]:
# TeacherAgent 생성
TEACHER_INSTRUCTIONS = """너는 답변을 평가하는 에이전트야. 답변이 일정, 비용, 취향 등 다양한 조건에 대한 고려를 했다면 [COMPLETE]이라고 대답해줘. 아니라면, COMPLETE을 표시하지 말고, 수정을 요청해줘.

평가 기준:
1. 일정: 구체적인 날짜, 시간, 기간이 포함되었는가?
2. 비용: 예산, 가격, 비용 정보가 포함되었는가?
3. 취향: 사용자의 선호도나 스타일을 고려했는가?
4. 실용성: 실제로 실행 가능한 계획인가?
5. 완성도: 모든 필요한 정보가 포함되었는가?

응답 형식:
평가 완료 시: "[COMPLETE] 모든 조건이 충족되었습니다."
개선 필요 시: "다음 사항을 보완해주세요: [구체적인 피드백]"

중요: [COMPLETE]는 모든 기준이 충족되었을 때만 사용합니다."""

agent_teacher = client.agents.create_agent(
    model="gpt-5.1",
    name="TeacherAgent",
    instructions=TEACHER_INSTRUCTIONS
)

print(f"✅ TeacherAgent 생성 완료!")
print(f"   ID: {agent_teacher.id}")

In [ ]:
# 생성된 에이전트 목록 확인
print("=" * 80)
print("워크플로우용 에이전트 목록")
print("=" * 80)

agents = client.agents.list_agents()
workflow_agents = ["TravelPlannerAgent", "LocalAgent", "TravelSummaryAgent", "StudentAgent", "TeacherAgent"]

for agent in agents:
    if agent.name in workflow_agents:
        tools = "None"
        if agent.tools:
            tools = ", ".join([t.type if hasattr(t, 'type') else str(t) for t in agent.tools])
        print(f"\n📌 {agent.name}")
        print(f"   ID: {agent.id}")
        print(f"   Model: {agent.model}")
        print(f"   Tools: {tools}")

print("\n" + "=" * 80)
print("\n💡 이제 Azure Portal에서 워크플로우를 생성하세요:")
print("   https://ai.azure.com > Build > Workflows > + Create workflow")
print("\n   Sequential Workflow:")
print("     Step 1: TravelPlannerAgent")
print("     Step 2: LocalAgent") 
print("     Step 3: TravelSummaryAgent")
print("\n   Group Chat Workflow:")
print("     Participants: StudentAgent, TeacherAgent")
print("     Termination: [COMPLETE] 포함 시")

### 워크플로우 호출 예제

In [ ]:
# Sequential Workflow 호출 예제
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ResponseStreamEventType

WORKFLOW_NAME = "Sequential-Workflow"  # ⚠️ 포털에서 생성한 워크플로우 이름
WORKFLOW_VERSION = "1"

# AI Project 클라이언트 생성
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(),
)

with project_client:
    workflow = {
        "name": WORKFLOW_NAME,
        "version": WORKFLOW_VERSION,
    }
    
    # OpenAI 클라이언트 가져오기
    openai_client = project_client.get_openai_client()

    # 대화 생성
    conversation = openai_client.conversations.create()
    print(f"Created conversation (id: {conversation.id})")

    # 워크플로우 호출 (스트리밍)
    print(f"\nCalling workflow: {WORKFLOW_NAME}...\n")
    stream = openai_client.responses.create(
        conversation=conversation.id,
        extra_body={"agent": {"name": workflow["name"], "type": "agent_reference"}},
        input="제주도 2박 3일 여행 일정 짜줘",
        stream=True,
        metadata={"x-ms-debug-mode-enabled": "1"},
    )

    # 스트리밍 이벤트 처리
    for event in stream:
        if event.type == ResponseStreamEventType.RESPONSE_OUTPUT_TEXT_DONE:
            print("\t", event.text)
        elif event.type == ResponseStreamEventType.RESPONSE_OUTPUT_ITEM_ADDED and event.item.type == "workflow_action":
            print(f"\n{'='*60}")
            print(f"Actor - '{event.item.action_id}':")
            print(f"{'='*60}")
        elif event.type == ResponseStreamEventType.RESPONSE_OUTPUT_ITEM_DONE and event.item.type == "workflow_action":
            print(f"\n✓ Workflow Item '{event.item.action_id}' is '{event.item.status}'")
            print(f"  (previous item was: '{event.item.previous_action_id}')")
        elif event.type == ResponseStreamEventType.RESPONSE_OUTPUT_TEXT_DELTA:
            print(event.delta, end="", flush=True)

    # 정리
    print("\n\n✅ Workflow completed!")
    openai_client.conversations.delete(conversation_id=conversation.id)
    print("Conversation deleted")

## 워크플로우 생성

**⚠️ 중요**: 워크플로우는 현재 Azure Portal에서만 생성 가능합니다.

### Azure Portal에서 워크플로우 생성 방법:

1. [Azure AI Foundry](https://ai.azure.com) 접속
2. **Build > Workflows** 메뉴
3. **+ Create workflow** 클릭
4. 워크플로우 타입 선택:
   - Sequential Workflow
   - Group Chat Workflow  
   - Human-in-loop Workflow

### 워크플로우 구성 예시:

**Sequential Workflow (여행 계획)**
```
User Input → SearchAgent → PlannerAgent → ReviewAgent → Output
```

**Group Chat Workflow (학습 토론)**
```
User Question → StudentAgent ↔ TeacherAgent → Consensus
```

생성이 완료되면 아래 코드로 실행할 수 있습니다.

## 워크플로우 실행

포털에서 생성한 워크플로우를 Python 코드로 실행합니다.

In [ ]:
# 워크플로우 실행 코드
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ResponseStreamEventType

# 클라이언트 생성
credential = DefaultAzureCredential()
project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

# ⚠️ 워크플로우 이름과 버전 설정 (포털에서 생성한 것으로 변경)
WORKFLOW_NAME = "Sequential-Workflow"  # ⚠️ 변경 필요
WORKFLOW_VERSION = "1"

with project_client:
    # OpenAI 클라이언트 가져오기
    openai_client = project_client.get_openai_client()
    
    # Conversation 생성
    conversation = openai_client.conversations.create()
    print(f"✅ Conversation 생성: {conversation.id}")
    
    # 워크플로우 실행 (스트리밍)
    print(f"\n🚀 워크플로우 실행 중: {WORKFLOW_NAME}...\n")
    print("=" * 80)
    
    stream = openai_client.responses.create(
        conversation=conversation.id,
        extra_body={"agent": {"name": WORKFLOW_NAME, "type": "agent_reference"}},
        input="제주도 2박 3일 여행 일정 짜줘",  # ⚠️ 원하는 질문으로 변경
        stream=True,
        metadata={"x-ms-debug-mode-enabled": "1"}
    )
    
    # 스트리밍 결과 처리
    for event in stream:
        if event.type == ResponseStreamEventType.RESPONSE_OUTPUT_TEXT_DELTA:
            print(event.delta, end="", flush=True)
        elif event.type == ResponseStreamEventType.RESPONSE_OUTPUT_ITEM_ADDED and event.item.type == "workflow_action":
            print(f"\n\n🤖 Actor: {event.item.action_id}")
        elif event.type == ResponseStreamEventType.RESPONSE_OUTPUT_ITEM_DONE and event.item.type == "workflow_action":
            print(f"\n✅ '{event.item.action_id}' 완료 (상태: {event.item.status})")
    
    print("\n" + "=" * 80)
    print("\n✅ 워크플로우 실행 완료!")
    
    # Conversation 삭제
    openai_client.conversations.delete(conversation_id=conversation.id)
    print(f"🗑️ Conversation 삭제됨")

### 워크플로우 설계

**포털에서 구성:**

```
TravelPlannerAgent → [사용자 승인] → LocalAgent → TravelSummaryAgent
```

**Approval 설정:**
- Approval message: "생성된 여행 계획을 검토해주세요. 승인하시겠습니까?"
- Options: Approve / Reject / Modify
- Timeout: 24시간

### 💡 Human-in-loop 모범 사례

**권장사항:**
- 승인 지점을 명확히 표시
- 타임아웃 설정으로 무한 대기 방지
- 사용자에게 컨텍스트 제공 (이전 대화 요약)
- 간단한 승인 옵션 제공 (예/아니오/수정)

**피해야 할 것:**
- 너무 많은 승인 지점
- 불명확한 승인 질문
- 긴 타임아웃 (사용자 경험 저하)
- 승인 후 되돌리기 불가능한 구조

## 📚 추가 리소스

- [Microsoft Foundry Workflows 개요](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/concepts/workflow?view=foundry)
- [Microsoft Agent Framework Workflows Orchestrations 패턴](https://learn.microsoft.com/en-us/agent-framework/user-guide/workflows/orchestrations/overview)

## 다음 단계

복잡한 워크플로우를 구축했습니다! 이제 에이전트와 워크플로우의 성능을 평가하는 방법을 학습합니다:

➡️ **[06. 평가](./06-evaluations.ipynb)**: 에이전트 및 워크플로우의 품질을 체계적으로 평가합니다.